## Dynamic Programming: Policy Iteration

Task: Implement policy iteration in 4x4 GridWorld

```
===========
T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T
===========

T: target
x: current position of agent
o: empty
```

In [1]:
import numpy as np
import pprint
import sys

from envs.gridworld import GridworldEnv

To get familiar with the GridWorld environment, let's explore the world with random policy.

We can do the action in the environment with `env.step` function.

For example:
```python
observation, reward, done, info = env.step(action)
```

In [2]:
env = GridworldEnv()
directions = ['UP', 'RIGHT', 'DOWN', 'LEFT']
discount_factor = 1.0
theta = 0.00001

In [3]:
random_policy = np.ones([env.nS, env.nA]) / env.nA


current_state = env.s = 6  # Fix start point

print('Start Position: ({}, {})'.format(
        current_state // env.shape[1], current_state % env.shape[1]))
action = np.argmax(random_policy[current_state])
print('Number of States: {}'.format(env.nS))
print('Number of Actions: {}'.format(env.nA))
env._render() # print current environment
print()


observation, reward, done, info = env.step(action)

current_state = observation
print('After an Action: {}'.format(directions[action]))
env._render()


Start Position: (1, 2)
Number of States: 16
Number of Actions: 4
T  o  o  o
o  o  x  o
o  o  o  o
o  o  o  T

After an Action: UP
T  o  x  o
o  o  o  o
o  o  o  o
o  o  o  T


### Policy Evaluation

Policy evaluation produces **value function** for every states **from given policy**.

In [4]:
def policy_evaluation(env, policy):
    V = np.zeros(env.nS)

    while True:
        delta = 0
        for s in range(env.nS):
            v = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P[s][a]:
                    v += action_prob * prob * \
                        (reward + discount_factor * V[next_state])

            delta = max(delta, np.abs(v - V[s]))
            V[s] = v

        if delta < theta:
            break

    return np.array(V)

In [5]:
def test_policy_evaluation():
    # Evaluate random policy

    random_policy = np.ones([env.nS, env.nA]) / env.nA
    v = policy_evaluation(env, random_policy)

    print("Value Function:")
    print(v.reshape(env.shape))
    print("")

In [6]:
test_policy_evaluation()

Value Function:
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]



### (Greedy) Policy Improvement

Now, let's implement greedy-style policy improvement for actually updating to the better policy, that chooses the action that gives the maximum value currently.

![image.png](https://www.dropbox.com/s/tg8rzuelo4besk6/policy_iteration.png?dl=1)

`one_step_lookahead` function calculates the value of each action using the value function from policy evaluation.

In [14]:
def policy_improvement(env, policy):
        def one_step_lookahead(state, V):
            A = np.zeros(env.nA)
            for a in range(env.nA):
                for prob, next_state, reward, done in env.P[state][a]:
                    A[a] += prob * \
                        (reward + discount_factor * V[next_state])
            return A

        V = policy_evaluation(env, policy)

        policy_stable = True

        for s in range(env.nS):
            chosen_a = np.argmax(policy[s])

            action_values = one_step_lookahead(s, V)

            # Use list: breaking the tie
            max_indices = []
            max_value = -99999
            for i, value in enumerate(action_values):
                if value == max_value:
                    max_indices.append(i)
                elif value > max_value:
                    max_indices = [i]
                    max_value = value

            improved_policy = [0] * env.nA
            prob = 1 / len(max_indices)
            for i in max_indices:
                improved_policy[i] = prob

            policy[s] = improved_policy

            best_a = np.argmax(action_values)

            if chosen_a not in max_indices:
                policy_stable = False

        return policy, policy_stable

In [12]:
def test_policy_improvement():
    policy_stable = False
    
    # Start from random policy!
    policy = random_policy
    
    # Until the policy doesn't improve anymore (converge)
    while not policy_stable:
        policy, policy_stable = policy_improvement(env, policy)

    print("Policy Probability Distribution:")
    print(policy)
    print("")

    print("Policy (0=up, 1=right, 2=down, 3=left):")
    print(np.reshape(np.argmax(policy, axis=1), env.shape))
    print("")

In [15]:
test_policy_improvement()

Policy Probability Distribution:
[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.5  0.5 ]
 [1.   0.   0.   0.  ]
 [0.5  0.   0.   0.5 ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.5  0.5  0.  ]
 [0.   0.   1.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]

Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]



Now Let's simulate that the agent can explore the gridworld more efficiently by the improved policy!

In [19]:
def get_action(state, policy):
    return np.argmax(policy[state])

def run_policy_iteration(env):
    policy_stable = False
    
    policy = random_policy
    while not policy_stable:
        policy, policy_stable = policy_improvement(env, policy)
        
    return policy
        
env = GridworldEnv()
current_state = env.s 

print('Start Position: ({}, {})'.format(
    current_state // env.shape[1], current_state % env.shape[1]))

env._render()

policy = run_policy_iteration(env)

done = False
while not done:
    action = get_action(current_state, policy)
    observation, reward, done, info = env.step(action)

    current_state = observation
    print('Action: {}'.format(directions[action]))
    env._render()

Start Position: (2, 1)
T  o  o  o
o  o  o  o
o  x  o  o
o  o  o  T
Action: UP
T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
Action: UP
T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T
Action: LEFT
x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T
